## Apartado 1 - Clasificación de texto según el autor
Construid dos modelos de diferentes autores: Josep Carner y Miquel dels Sants Oliver. Para ello, podéis usar sus obras disponibles en el Proyecto Gutenberg. Por ejemplo, de Carner, tomad la traducción de los cuentos de Mark Twain. Luego, clasificad frases en el estilo de cada uno que muestren cómo vuestro modelo las identifica correctamente.

In [1]:
from my_utils import open_data
from text import *

hostal = open_data("CA-text/hostal.txt").read()
wordseq = words(hostal)

P_Miquel = UnigramWordModel(wordseq, 5)

adventures = open_data("CA-text/adventures.txt").read()
wordseq = words(adventures)

P_Josep = UnigramWordModel(wordseq, 5)

In [2]:
from learning import NaiveBayesLearner

dist = {('Miquel', 1): P_Miquel, ('Josep', 1): P_Josep}

nBS = NaiveBayesLearner(dist, simple=True)

/Users/davidramirez/Documents/IEDIB/02 Modelos IA/T5 NLP/.venv/lib/python3.13/site-packages/qpsolvers/solvers/__init__.py:752: UserWarning: no QP solver found on your system, you can install solvers from PyPI by ``pip install qpsolvers[open_source_solvers]``
  warnings.warn(


In [3]:
def recognize(sentence, nBS):
    sentence = sentence.lower()
    sentence_words = words(sentence)
    
    return nBS(sentence_words)

Primero he probado a usar una frase extraída de los archivos CA-text/hostal.txt y CA-text/adventures.txt para comprobar si el modelo es capaz de identificar correctamente el autor de la frase.

In [4]:
recognize("havia donat la direcció a un personatge destinat a fer molt de renou", nBS)

'Miquel'

In [5]:
recognize("potser mai no bastarà mentre el món sigui món", nBS)

'Josep'

Para probar con otro ejemplo, le he preguntado al ChatGPT la diferencia de estilo entre los dos autores. A continuación, se muestra la respuesta del ChatGPT:

```
Diferencias resumidas:
	•	Josep Carner → Poètic, refinat, musical i idealitzat.
	•	Miquel dels Sants Oliver → Narratiu, directe, reflexiu i analític.
```

A raíz de esto he probado a clasificar dos frases de cada autor para ver si el modelo es capaz de diferenciar el estilo de cada uno. Pero en este caso estas frases no han sido sacadas de los archivios .txt sino generadas.

In [6]:
recognize("Els carrers, encara humits, semblaven desertar de la nit", nBS)

'Miquel'

In [7]:
recognize("Amb un gest majestuós em fità un instant", nBS)

'Josep'

## Apartado 2 - Generar texto

Construye modelos n-gram con n=1, n=3, n=5 y n=7 a partir de un texto de tu elección. Puede ser una obra del Proyecto Gutenberg o una noticia de prensa, por ejemplo. Observa cómo, a medida que aumenta el número de palabras consideradas, la plausibilidad del texto generado mejora.

He optado por elegir Don Quijote de la Mancha como corpus de entranmiento para el modelo. El texto está guardado en /aima-data/ES-text/quijote.txt

In [224]:
import random
from text import NgramWordModel

def generar_texto(model, n, longitud):
    generated_text = []
    # Elegimos una semilla aleatoria del modelo
    seed = random.choice(list(model.dictionary.keys()))
    generated_text.extend(seed)
    
    for i in range(longitud - n):
        last_sequence = tuple(generated_text[-(n-1):])  # Tomamos los últimos caracteres generados
        next_possibilities = []  # Lista para guardar opciones

        for k in model.dictionary.keys():  # Revisamos todos los n-gramas aprendidos
            if k[:n-1] == last_sequence:  # Si el inicio del n-grama coincide...
                next_possibilities.append(k[n-1])  # Guardamos la siguiente letra
        
        if not next_possibilities:  # Si no hay opciones, salimos
            break

        next = random.choice(next_possibilities)  # Elegimos una opción aleatoria
        generated_text.append(next)  # La añadimos al texto generado
    
    return ' '.join(generated_text)

# Cargar texto y entrenar el modelo
quijote = open_data("ES-text/quijote.txt").read()
wordseq = words(quijote)

Una vez preparada la función hago distintas pruebas modificando la n. Y veremos que es correcta la afirmación anterior; a medida que aumentamos el número de carácteres que se tienen en cuenta el texto generado empieza a tener más sentido.

In [243]:
model_ngram = NgramWordModel(3, wordseq)

# Generar texto
new_text = generar_texto(model_ngram, 3, 50)
print(new_text)

dila dijo don juan hab a adelantado sab an que era podenco mi perro y dando consejos a quien iba cap tulo primero que habl is de darme el gobierno los deseos que duermen y mueren en sus contentos no cumpl a cuanto se llegan a los encadenados soltar los


In [244]:
model_ngram = NgramWordModel(5, wordseq)

# Generar texto
new_text = generar_texto(model_ngram, 5, 50)
print(new_text)

mire vuesa merced lo que hace se or don quijote ni quiera llevar las cosas tan por los cabos que no querr que se aprecie lo que montare la renta de la tal nsula y se descuente de mi salario gata por cantidad sancho amigo respondi don quijote ni es


In [252]:
model_ngram = NgramWordModel(7, wordseq)

# Generar texto
new_text = generar_texto(model_ngram, 7, 50)
print(new_text)

quise poner el negocio en aventura y as los hice rescatar por la misma orden que yo me rescat entregando todo el dinero al mercader para que con certeza y seguridad pudiese hacer la fianza al cual nunca descubrimos nuestro trato y secreto por el peligro que hab a cap


Por hacer una prueba con una n mucho más grande he decido usar 30. Vemos que da un buen resultado al tener más contexto.  
Y a pesar de que algunas frases de las que genera no tienen un sentido a nivel de significado, por lo menos sí que son correctas en gran mayoría gramaticalmente.

In [250]:
model_ngram = NgramWordModel(30, wordseq)

# Generar texto
new_text = generar_texto(model_ngram, 30, 50)
print(new_text)

ni con mis obras jam s quise ni supe ofenderte luego no es verdad dijo claudia que ibas esta ma ana a desposarte con leonora la hija del rico balvastro no por cierto respondi don vicente mi mala fortuna te debi de llevar estas nuevas para que celosa me quitases


## Apartado 3 - Análisis sintáctico

Aplicad las herramientas de alguna libería de Python para analizar sintácticamente la oración siguiente:  

```
Todos los seres humanos nacen libres e iguales en dignidad y derechos.
```